# Rightmove - Building a dataset of properties available for renting in London

---

This code scrapes property listings available for rent from Rightmove.co.uk, the UK's largest property listing website. This notebook will create a csv file with the following information for each property listing:

*   Borough name
*   Monthly rent
*   Property link
*   Number of bedrooms
*   Address of property

This code is a modified version of this original code by Brandon Low.
https://github.com/BrandonLow96/Rightmove-scrapping

In [32]:
# importing our libraries

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import random

"""
Rightmove uses specific codes to describe each London borough, I have 
manually collected these codes by search for each borough individually.
"""

BOROUGHS = {
    "City of London": "5E61224",
    "Barking and Dagenham": "5E61400",
    "Barnet": "5E93929",
    "Bexley": "5E93932",
    "Brent": "5E93935",
    "Bromley": "5E93938",
    "Camden": "5E93941",
    "Croydon": "5E93944",
    "Ealing": "5E93947",
    "Enfield": "5E93950",
    "Greenwich": "5E61226",
    "Hackney": "5E93953",
    "Hammersmith and Fulham": "5E61407",
    "Haringey": "5E61227",
    "Harrow": "5E93956",
    "Havering": "5E61228",
    "Hillingdon": "5E93959",
    "Hounslow": "5E93962",
    "Islington": "5E93965",
    "Kensington and Chelsea": "5E61229",
    "Kingston upon Thames": "5E93968",
    "Lambeth": "5E93971",
    "Lewisham": "5E61413",
    "Merton": "5E61414",
    "Newham": "5E61231",
    "Redbridge": "5E61537",
    "Richmond upon Thames": "5E61415",
    "Southwark": "5E61518",
    "Sutton": "5E93974",
    "Tower Hamlets": "5E61417",
    "Waltham Forest": "5E61232",
    "Wandsworth": "5E93977",
    "Westminster": "5E93980",
}

In [39]:

def main():

    # initialise index, this tracks the page number we are on. every additional page adds 24 to the index

    # create lists to store our data
    all_apartment_links = []
    all_description = []
    all_address = []
    all_price = []
    boroughs = []

    # apparently the maximum page limit for rightmove is 42
    for borough in list(BOROUGHS.values()):

        # initialise index, this tracks the page number we are on. every additional page adds 24 to the index
        index = 0

        key = [key for key, value in BOROUGHS.items() if value == borough]
        print(f"We are scraping the borough named: {key}")
        for pages in range(41):

            # define our user headers
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"
            }

            # the website changes if the you are on page 1 as compared to other pages
            if index == 0:
                rightmove = f"https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%{borough}&sortType=6&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="

            elif index != 0:
                rightmove = f"https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%{borough}&sortType=6&index={index}&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="

            # request our webpage
            res = requests.get(rightmove, headers=headers)

            # check status
            res.raise_for_status()

            soup = BeautifulSoup(res.text, "html.parser")

            # This gets the list of apartments
            apartments = soup.find_all("div", class_="l-searchResult is-list")

            # This gets the number of listings
            number_of_listings = soup.find(
                "span", {"class": "searchHeader-resultCount"}
            )
            number_of_listings = number_of_listings.get_text()
            number_of_listings = int(number_of_listings.replace(",", ""))

            for i in range(len(apartments)):

                # tracks which apartment we are on in the page
                apartment_no = apartments[i]

                # append link
                apartment_info = apartment_no.find("a", class_="propertyCard-link")
                link = "https://www.rightmove.co.uk" + apartment_info.attrs["href"]
                all_apartment_links.append(link)

                # append address
                address = (
                    apartment_info.find("address", class_="propertyCard-address")
                    .get_text()
                    .strip()
                )
                all_address.append(address)

                # append description
                description = (
                    apartment_info.find("h2", class_="propertyCard-title")
                    .get_text()
                    .strip()
                )
                all_description.append(description)

                # append price
                price = (
                    apartment_no.find("span", class_="propertyCard-priceValue")
                    .get_text()
                    .strip()
                )
                all_price.append(price)
                
                #append the name of the borough
                boroughs.append(key)
                

            print(f"You have scrapped {pages + 1} pages of apartment listings.")
            print(f"You have {number_of_listings - index} listings left to go")
            print("\n")

            # code to ensure that we do not overwhelm the website
            time.sleep(random.randint(1, 3))

            # Code to count how many listings we have scrapped already.
            index = index + 24

            if index >= number_of_listings:
                break

    # convert data to dataframe
    data = {
        "Links": all_apartment_links,
        "Address": all_address,
        "Description": all_description,
        "Price": all_price,
        "Borough":boroughs
    }
    df = pd.DataFrame.from_dict(data)
    df.to_csv(r"rents_data_with_boroughs.csv", encoding="utf-8", header="true", index = False)


if __name__ == "__main__":
    main()

We are scraping the borough named: ['City of London']
You have scrapped 1 pages of apartment listings.
You have 103 listings left to go


You have scrapped 2 pages of apartment listings.
You have 79 listings left to go


You have scrapped 3 pages of apartment listings.
You have 55 listings left to go


You have scrapped 4 pages of apartment listings.
You have 31 listings left to go


You have scrapped 5 pages of apartment listings.
You have 7 listings left to go


We are scraping the borough named: ['Barking and Dagenham']
You have scrapped 1 pages of apartment listings.
You have 67 listings left to go


You have scrapped 2 pages of apartment listings.
You have 43 listings left to go


You have scrapped 3 pages of apartment listings.
You have 19 listings left to go


We are scraping the borough named: ['Barnet']
You have scrapped 1 pages of apartment listings.
You have 643 listings left to go


You have scrapped 2 pages of apartment listings.
You have 619 listings left to go


You have

You have scrapped 32 pages of apartment listings.
You have 496 listings left to go


You have scrapped 33 pages of apartment listings.
You have 472 listings left to go


You have scrapped 34 pages of apartment listings.
You have 448 listings left to go


You have scrapped 35 pages of apartment listings.
You have 424 listings left to go


You have scrapped 36 pages of apartment listings.
You have 392 listings left to go


You have scrapped 37 pages of apartment listings.
You have 368 listings left to go


You have scrapped 38 pages of apartment listings.
You have 344 listings left to go


You have scrapped 39 pages of apartment listings.
You have 320 listings left to go


You have scrapped 40 pages of apartment listings.
You have 304 listings left to go


You have scrapped 41 pages of apartment listings.
You have 273 listings left to go


We are scraping the borough named: ['Croydon']
You have scrapped 1 pages of apartment listings.
You have 422 listings left to go


You have scrapped 2

You have scrapped 13 pages of apartment listings.
You have 543 listings left to go


You have scrapped 14 pages of apartment listings.
You have 519 listings left to go


You have scrapped 15 pages of apartment listings.
You have 497 listings left to go


You have scrapped 16 pages of apartment listings.
You have 471 listings left to go


You have scrapped 17 pages of apartment listings.
You have 447 listings left to go


You have scrapped 18 pages of apartment listings.
You have 423 listings left to go


You have scrapped 19 pages of apartment listings.
You have 401 listings left to go


You have scrapped 20 pages of apartment listings.
You have 375 listings left to go


You have scrapped 21 pages of apartment listings.
You have 351 listings left to go


You have scrapped 22 pages of apartment listings.
You have 327 listings left to go


You have scrapped 23 pages of apartment listings.
You have 303 listings left to go


You have scrapped 24 pages of apartment listings.
You have 279 li

You have scrapped 3 pages of apartment listings.
You have 1678 listings left to go


You have scrapped 4 pages of apartment listings.
You have 1653 listings left to go


You have scrapped 5 pages of apartment listings.
You have 1629 listings left to go


You have scrapped 6 pages of apartment listings.
You have 1605 listings left to go


You have scrapped 7 pages of apartment listings.
You have 1582 listings left to go


You have scrapped 8 pages of apartment listings.
You have 1558 listings left to go


You have scrapped 9 pages of apartment listings.
You have 1533 listings left to go


You have scrapped 10 pages of apartment listings.
You have 1509 listings left to go


You have scrapped 11 pages of apartment listings.
You have 1485 listings left to go


You have scrapped 12 pages of apartment listings.
You have 1462 listings left to go


You have scrapped 13 pages of apartment listings.
You have 1437 listings left to go


You have scrapped 14 pages of apartment listings.
You have 14

You have scrapped 13 pages of apartment listings.
You have 57 listings left to go


You have scrapped 14 pages of apartment listings.
You have 33 listings left to go


You have scrapped 15 pages of apartment listings.
You have 9 listings left to go


We are scraping the borough named: ['Newham']
You have scrapped 1 pages of apartment listings.
You have 266 listings left to go


You have scrapped 2 pages of apartment listings.
You have 242 listings left to go


You have scrapped 3 pages of apartment listings.
You have 218 listings left to go


You have scrapped 4 pages of apartment listings.
You have 194 listings left to go


You have scrapped 5 pages of apartment listings.
You have 170 listings left to go


You have scrapped 6 pages of apartment listings.
You have 145 listings left to go


You have scrapped 7 pages of apartment listings.
You have 121 listings left to go


You have scrapped 8 pages of apartment listings.
You have 97 listings left to go


You have scrapped 9 pages of apa

You have scrapped 28 pages of apartment listings.
You have 220 listings left to go


You have scrapped 29 pages of apartment listings.
You have 196 listings left to go


You have scrapped 30 pages of apartment listings.
You have 176 listings left to go


You have scrapped 31 pages of apartment listings.
You have 148 listings left to go


You have scrapped 32 pages of apartment listings.
You have 125 listings left to go


You have scrapped 33 pages of apartment listings.
You have 101 listings left to go


You have scrapped 34 pages of apartment listings.
You have 80 listings left to go


You have scrapped 35 pages of apartment listings.
You have 53 listings left to go


You have scrapped 36 pages of apartment listings.
You have 28 listings left to go


You have scrapped 37 pages of apartment listings.
You have 4 listings left to go


We are scraping the borough named: ['Waltham Forest']
You have scrapped 1 pages of apartment listings.
You have 149 listings left to go


You have scrapped